# Start Flux

In [2]:
# @markdown # Start 💕
# @markdown Click to download Model and Libraries

from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

clear_output()

%cd /content
!git clone -b totoro4 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2 gradio==3.50.2 python-multipart==0.0.12
!apt -y install -qq aria2

clear_output()

button = widgets.Button(description="Done! <3", button_style="success")
display(button)


Button(button_style='success', description='Done! <3', style=ButtonStyle())

---

In [ ]:
# @markdown # LoRA and Flux Models ✨
# @markdown Enter the name of the Hugging Face repository from which the model is loaded.

model_repo = "camenduru/FLUX.1-dev"  # @param {type:"string"}
# @markdown Enter the name of the LoRA on the Hugging Face from which the model is loaded
lora_model = "flux_realism_lora"     # @param {type:"string"}

# @markdown Enter the Hugging Face token to access private models. (not work sowwy)
hf_token = ""  # @param {type:"string"}

import os
import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro_extras import nodes_flux
from totoro import model_management
import gradio as gr
from IPython.display import clear_output

def download_with_hf_token(url, token, directory, output_name):
    if "huggingface.co" in url:
        url = url.replace("?download=true", "")
        if "/blob/" in url:
            url = url.replace("/blob/", "/resolve/")
        user_header = f'\"Authorization: Bearer {token}\"'
        os.system(f'aria2c --console-log-level=error --header={user_header} -c -x 16 -s 16 -k 1M {url} -d {directory} -o {output_name}')

checkpoint_dir = "/content/TotoroUI/models/checkpoints"
lora_dir = "/content/TotoroUI/models/loras"
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(lora_dir, exist_ok=True)

checkpoint_url = f"https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8-all-in-one.safetensors"
lora_url = f"https://huggingface.co/{model_repo}/resolve/main/{lora_model}.safetensors"

download_with_hf_token(checkpoint_url, hf_token, checkpoint_dir, "flux1-dev-fp8-all-in-one.safetensors")
download_with_hf_token(lora_url, hf_token, lora_dir, f"{lora_model}.safetensors")

CheckpointLoaderSimple = NODE_CLASS_MAPPINGS["CheckpointLoaderSimple"]()
LoraLoader = NODE_CLASS_MAPPINGS["LoraLoader"]()
FluxGuidance = nodes_flux.NODE_CLASS_MAPPINGS["FluxGuidance"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    unet, clip, vae = CheckpointLoaderSimple.load_checkpoint("flux1-dev-fp8-all-in-one.safetensors")
def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

@torch.inference_mode()
def generate(positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip):
    global unet, clip

    clear_output()

    if seed == 0:
        seed = random.randint(0, 18446744073709551615)

    unet_lora, clip_lora = LoraLoader.load_lora(unet, clip, f"{lora_model}.safetensors", lora_strength_model, lora_strength_clip)

    cond, pooled = clip_lora.encode_from_tokens(clip_lora.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    cond = FluxGuidance.append(cond, guidance)[0]

    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet_lora, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet_lora, scheduler, steps, 1.0)[0]
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    decoded = VAEDecode.decode(vae, sample)[0].detach()

    output_path = "/content/flux.png"
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(output_path)

    return output_path

with gr.Blocks(analytics_enabled=False) as demo:
    with gr.Row():
        with gr.Column():
            positive_prompt = gr.Textbox(lines=3, interactive=True, value="A small kitten sits next to a palm tree on a sunny beach, the kitten is about 3 months old, it has light grey fur, its eyes are very large and blue, it looks very playful and cute, around it are a few flowers and palm leaves, the sun's rays shine through the palm tree, creating a very idyllic atmosphere, there are no people around, only the natural landscape of the beach, the kitten looks very alone, but very content. Below it in the sand is written 'by: mikus', and slightly below that 'code: camenduru', a little further away from the text lies a single piece of seaweed, which looks very unusual.", label="Prompt")
            width = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="width")
            height = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="height")
            seed = gr.Slider(minimum=0, maximum=18446744073709551615, value=0, step=1, label="seed (0=random)")
            steps = gr.Slider(minimum=4, maximum=50, value=20, step=1, label="steps")
            guidance = gr.Slider(minimum=0, maximum=20, value=3.5, step=0.5, label="guidance")
            lora_strength_model = gr.Slider(minimum=0, maximum=1, value=1.0, step=0.1, label="lora_strength_model")
            lora_strength_clip = gr.Slider(minimum=0, maximum=1, value=1.0, step=0.1, label="lora_strength_clip")
            sampler_name = gr.Dropdown(["euler", "heun", "heunpp2", "dpm_2", "lms", "ddim"], label="sampler_name", value="euler")
            scheduler = gr.Dropdown(["normal", "sgm_uniform", "simple", "ddim_uniform"], label="scheduler", value="simple")
            generate_button = gr.Button("Generate")
        with gr.Column():
            output_image = gr.Image(label="Generated image", interactive=False)

    generate_button.click(fn=generate, inputs=[positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip], outputs=output_image)

demo.queue().launch(inline=False, share=True, debug=True)

---

# Tutorial